# Micro-SAM Training from OMERO Data

Train micro-SAM models using annotation tables from OMERO with automated data preparation.

## 1. Setup

In [2]:
# Import the package with training convenience functions
from omero_annotate_ai import (
    create_omero_connection_widget,
    create_training_data_widget,
    prepare_training_data_from_table,
    setup_training,    # Convenience function for training setup
    run_training       # Convenience function for training execution
)

# Additional imports
import datetime
from pathlib import Path
import torch

print(f"Available functions: Connection, Training Data, Training Utils")
#print(f"Device available: {'CUDA' if torch.cuda.is_available() else 'CPU'}")


Available functions: Connection, Training Data, Training Utils


## 2. OMERO Connection

In [ ]:
# Create and display OMERO connection widget
conn_widget = create_omero_connection_widget()
conn_widget.display()

📄 Loaded configuration from connection history: root@localhost
🔐 Password loaded from keychain (no expiration)


In [4]:
# Get the OMERO connection
conn = conn_widget.get_connection()

if conn is None:
    raise ConnectionError("No OMERO connection established.")

print(f"Connected to OMERO as: {conn.getUser().getName()}")

Connected to OMERO as: root


## 3. Training Data Selection

In [ ]:
# Create training data selection widget
training_widget = create_training_data_widget(connection=conn)
training_widget.display()

In [9]:
# Get selected training table
selected_table_id = training_widget.get_selected_table_id()
selected_table_info = training_widget.get_selected_table_info()

if selected_table_id:
    print(f"Selected training table:")
    print(f"  Table ID: {selected_table_id}")
    print(f"  Table Name: {selected_table_info.get('name', 'Unknown')}")
    print(f"  Created: {selected_table_info.get('created', 'Unknown')}")
else:
    raise ValueError("No training table selected. Please select a table above.")

Selected training table:
  Table ID: 1508
  Table Name: micro_sam_training_micro_sam_foci_test_20250825_201655
  Created: Unknown


## 4. Setup Training Directory

In [10]:
# Create output directory for training
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
home_dir = Path.home()
models_dir = home_dir / "micro-sam_models"
models_dir.mkdir(exist_ok=True)

folder_name = f"micro-sam-{timestamp}"
output_directory = models_dir / folder_name
output_directory.mkdir(exist_ok=True)

print(f"Training output directory: {output_directory}")

Training output directory: C:\Users\Maarten\micro-sam_models\micro-sam-20250825_203144


## 5. Automated Data Preparation

Use the automated data preparation function to download and organize training data.

In [11]:
# Run automated data preparation
try:
    training_result = prepare_training_data_from_table(
        conn=conn,
        table_id=selected_table_id,
        output_dir=output_directory,
        validation_split=0.2,  # 20% for validation
        clean_existing=True
    )
    
    print("\nTraining data preparation completed successfully!")
    print(f"\nDataset statistics:")
    for key, value in training_result['stats'].items():
        print(f"  {key}: {value}")
    
    # Store paths for later use in training
    training_input_dir = training_result['training_input']
    training_label_dir = training_result['training_label']
    val_input_dir = training_result['val_input']
    val_label_dir = training_result['val_label']
    
    print(f"\nDirectory structure created:")
    print(f"  Training images: {training_input_dir}")
    print(f"  Training labels: {training_label_dir}")
    print(f"  Validation images: {val_input_dir}")
    print(f"  Validation labels: {val_label_dir}")
    
except Exception as e:
    print(f"Error during data preparation: {e}")
    raise

Loaded table with 6 rows
Table saved to: C:\Users\Maarten\micro-sam_models\micro-sam-20250825_203144\table_1508.csv
✅ Using 6 processed rows for training
Optional columns found: ['is_volumetric']
Table schema validated for processing
Using 3 training images and 3 validation images


Preparing training data:   0%|          | 0/3 [00:00<?, ?it/s]INFO:omero.gateway:Registered b83f12d0-8571-4503-bca6-acbfbc6cb054/9a67932c-4c66-45bb-a0a2-9f5a3682c2d7omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000


Item 0 - Image ID: 459, Patch: False, Dimensions: 0x0 at (271,456), Volumetric: False
  2D Full Image Request - start_coords: (0, 0, 0, 0, 0)


INFO:omero.gateway:Unregistered b83f12d0-8571-4503-bca6-acbfbc6cb054/9a67932c-4c66-45bb-a0a2-9f5a3682c2d7omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000
Preparing training data:  33%|███▎      | 1/3 [00:00<00:01,  1.46it/s]INFO:omero.gateway:Registered b83f12d0-8571-4503-bca6-acbfbc6cb054/573be73d-5a37-4c74-85ec-115b14af9f64omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000


  Returned array shape: (1024, 1024, 1, 4, 1)
  Extracted 2D shape: (1024, 1024)
  Saved 2D TIFF to C:\Users\Maarten\micro-sam_models\micro-sam-20250825_203144\training_input\input_00000.tif with shape (1024, 1024)
  Attempting to download label with ID: 1502
  File annotation found: seg_00000.tif
  Label shape: (512, 512) saved to C:\Users\Maarten\micro-sam_models\micro-sam-20250825_203144\training_label\label_00000.tif
Item 1 - Image ID: 453, Patch: False, Dimensions: 0x0 at (386,454), Volumetric: False
  2D Full Image Request - start_coords: (0, 0, 0, 0, 0)


...

  Returned array shape: (1024, 1024, 1, 4, 1)
  Extracted 2D shape: (1024, 1024)
  Saved 2D TIFF to C:\Users\Maarten\micro-sam_models\micro-sam-20250825_203144\val_input\input_00000.tif with shape (1024, 1024)
  Attempting to download label with ID: 1505
  File annotation found: seg_00003.tif
  Label shape: (512, 512) saved to C:\Users\Maarten\micro-sam_models\micro-sam-20250825_203144\val_label\label_00000.tif
Item 1 - Image ID: 454, Patch: False, Dimensions: 0x0 at (356,470), Volumetric: False
  2D Full Image Request - start_coords: (0, 0, 0, 0, 0)


INFO:omero.gateway:Unregistered b83f12d0-8571-4503-bca6-acbfbc6cb054/f7164932-415b-438d-b6e7-5bad7ef27983omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000
Preparing val data:  67%|██████▋   | 2/3 [00:01<00:00,  1.56it/s]INFO:omero.gateway:Registered b83f12d0-8571-4503-bca6-acbfbc6cb054/96f5625c-e380-4163-937b-9111852c5171omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000


  Returned array shape: (1024, 1024, 1, 4, 1)
  Extracted 2D shape: (1024, 1024)
  Saved 2D TIFF to C:\Users\Maarten\micro-sam_models\micro-sam-20250825_203144\val_input\input_00001.tif with shape (1024, 1024)
  Attempting to download label with ID: 1506
  File annotation found: seg_00004.tif
  Label shape: (512, 512) saved to C:\Users\Maarten\micro-sam_models\micro-sam-20250825_203144\val_label\label_00001.tif
Item 2 - Image ID: 456, Patch: False, Dimensions: 0x0 at (425,94), Volumetric: False
  2D Full Image Request - start_coords: (0, 0, 0, 0, 0)


INFO:omero.gateway:Unregistered b83f12d0-8571-4503-bca6-acbfbc6cb054/96f5625c-e380-4163-937b-9111852c5171omero.api.RawPixelsStore -t -e 1.1:tcp -h 172.19.0.8 -p 34317 -t 60000
Preparing val data: 100%|██████████| 3/3 [00:01<00:00,  1.57it/s]

  Returned array shape: (1024, 1024, 1, 4, 1)
  Extracted 2D shape: (1024, 1024)
  Saved 2D TIFF to C:\Users\Maarten\micro-sam_models\micro-sam-20250825_203144\val_input\input_00002.tif with shape (1024, 1024)
  Attempting to download label with ID: 1507
  File annotation found: seg_00005.tif
  Label shape: (512, 512) saved to C:\Users\Maarten\micro-sam_models\micro-sam-20250825_203144\val_label\label_00002.tif
✅ Training data prepared successfully in: C:\Users\Maarten\micro-sam_models\micro-sam-20250825_203144
Statistics: {'n_training_images': 3, 'n_training_labels': 3, 'n_val_images': 3, 'n_val_labels': 3, 'total_rows_processed': 6}

Training data preparation completed successfully!

Dataset statistics:
  n_training_images: 3
  n_training_labels: 3
  n_val_images: 3
  n_val_labels: 3
  total_rows_processed: 6

Directory structure created:
  Training images: C:\Users\Maarten\micro-sam_models\micro-sam-20250825_203144\training_input
  Training labels: C:\Users\Maarten\micro-sam_models\

## 6. Micro-SAM Training Setup

Configure and run micro-SAM training using the prepared data.

In [12]:
# ✨ Setup training configuration using convenience function
training_config = setup_training(
    training_result,
    epochs=10,               # Primary parameter: number of epochs (use 50+ for real training)
    batch_size=1,            # Adjust based on GPU memory
    learning_rate=1e-5,      # Conservative learning rate
    patch_shape=(512, 512),  # Input patch size
    model_type="vit_b",       # SAM model variant
    n_objects_per_batch=25   # Objects per batch for sampling
)

print("Training configuration prepared!")
print(f'Model name: {training_config["model_name"]}')
print(f'Output directory: {training_config["output_dir"]}')
print(f'Training epochs: {training_config["epochs"]}')
print(f'Calculated iterations: {training_config["n_iterations"]}')


Training configuration prepared!
Model name: micro_sam_training_20250825_203206
Output directory: C:\Users\Maarten\micro-sam_models\micro-sam-20250825_203144
Training epochs: 10
Calculated iterations: 30


In [13]:
# ✨ Execute training with convenience function
print("Starting micro-SAM training...")

training_results = run_training(training_config, framework="microsam")

print(f'🎉 Training completed successfully!')
print(f'Training Results:')
print(f'  Model name: {training_results["model_name"]}')
print(f'  Final model: {training_results.get("final_model_path", "Not available")}')
print(f'  Checkpoints saved: {len(training_results.get("checkpoints", []))}')
print(f'  Output directory: {training_results["output_dir"]}')


Starting micro-SAM training...
Starting micro-SAM training...
Model name: micro_sam_training_20250825_203206
Model type: vit_b
Training configuration:
  Patch shape: (512, 512)
  Batch size: 1
  Learning rate: 1e-05
  Epochs: 10
  Objects per batch: 25
  Checkpoint folder: C:\Users\Maarten\micro-sam_models\micro-sam-20250825_203144\checkpoints
  Using patch shape: (1, 512, 512)
Training device: cpu


AssertionError: (3, 1024, 1024), (3, 512, 512)

## 7. Model Export and Summary

In [ ]:
# Find the best checkpoint
checkpoints = list(checkpoint_folder.glob("*.pt"))
if checkpoints:
    latest_checkpoint = sorted(checkpoints)[-1]
    print(f"Latest checkpoint: {latest_checkpoint}")
    
    # Export model for inference
    export_path = output_directory / f"{model_name}_final.pt"
    print(f"Model exported to: {export_path}")
else:
    print("No checkpoints found.")

print(f"\nTraining summary:")
print(f"  Output directory: {output_directory}")
print(f"  Model name: {model_name}")
print(f"  Training completed with {n_iterations} iterations")
print(f"  Dataset statistics: {training_result['stats']}")

## 8. Cleanup

In [ ]:
# Close OMERO connection
if conn is not None:
    conn.close()
    print("OMERO connection closed.")
else:
    print("No active OMERO connection to close.")